<a href="https://colab.research.google.com/github/nevasini/Breast-Cancer-Detection/blob/main/mobileNETV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
! unzip -d dataset "/content/drive/MyDrive/master-dataset.zip"

Streaming output truncated to the last 5000 lines.
  inflating: dataset/master-dataset/zero/12894_idx5_x2751_y1501_class0.png  
  inflating: dataset/__MACOSX/master-dataset/zero/._12894_idx5_x2751_y1501_class0.png  
  inflating: dataset/master-dataset/zero/9022_idx5_x601_y251_class0.png  
  inflating: dataset/__MACOSX/master-dataset/zero/._9022_idx5_x601_y251_class0.png  
  inflating: dataset/master-dataset/zero/12820_idx5_x2851_y1601_class0.png  
  inflating: dataset/__MACOSX/master-dataset/zero/._12820_idx5_x2851_y1601_class0.png  
  inflating: dataset/master-dataset/zero/10304_idx5_x1101_y901_class0.png  
  inflating: dataset/__MACOSX/master-dataset/zero/._10304_idx5_x1101_y901_class0.png  
  inflating: dataset/master-dataset/zero/9225_idx5_x1401_y551_class0.png  
  inflating: dataset/__MACOSX/master-dataset/zero/._9225_idx5_x1401_y551_class0.png  
  inflating: dataset/master-dataset/zero/12911_idx5_x1851_y801_class0.png  
  inflating: dataset/__MACOSX/master-dataset/zero/._12911_id

In [27]:
from keras.preprocessing.image import ImageDataGenerator
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

train_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory="/content/dataset/master-dataset",
                                                 shuffle=True,
                                                 target_size=(224,224), 
                                                 subset="training",
                                                 class_mode='categorical')

Found 222020 images belonging to 2 classes.


In [28]:
test_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory="/content/dataset/master-dataset",
                                                 shuffle=False,
                                                 target_size=(224,224), 
                                                 subset="validation",
                                                 class_mode='categorical')

Found 55504 images belonging to 2 classes.


In [42]:
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten,BatchNormalization
from keras.optimizers import Adam, Adagrad, Adadelta
import cv2
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

In [31]:
IMG_SHAPE = (224,224) + (3,)
tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    alpha=1.0,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",

)

14540800/14536120 [==============================] - 0s 0us/step


In [32]:
tf.keras.applications.MobileNetV2.trainable = True

In [33]:
from keras.applications import MobileNetV2

In [34]:
mobilenet = MobileNetV2(include_top =True, weights = None, input_shape=(224,224,3), pooling='average')

In [35]:
mobilenet.trainable= True

In [43]:
model=Sequential()

model.add(mobilenet)

model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation ='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [44]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 1000)              3538984   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               512512    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_12 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_13 (Dense)             (None, 2)                

In [45]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics= ['accuracy'])

In [46]:
fp = "/content/dataset/master-dataset.h5"
checkpoint = ModelCheckpoint(fp, monitor = 'val_loss', save_best_only=True, mode = 'min')
checkpoint

In [ ]:
model.fit(train_dataset, validation_data=test_dataset, epochs = 5, callbacks = checkpoint, shuffle = True, batch_size = 32)

Epoch 1/5
4657/6939 [===================>..........] - ETA: 13:13 - loss: 0.3526 - accuracy: 0.8479